In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import csv
import warnings
import json
import threading
from selenium_recaptcha import Recaptcha_Solver
warnings.filterwarnings("ignore")

    Crawl data

    URL = https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html

Khởi tạo đối tượng webdriver và khai báo link URL

In [2]:
#URL = https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html
productLinks = []
nameFile = "dataLinkAloNhaDat.txt"
nameFileRS = "dataLinkAloNhaDatResult.txt"

Function resolve capcha

In [3]:
def resolveCapcha(browser): 
    try:
        btn = browser.find_element(By.CLASS_NAME, 'verify')
        solver = Recaptcha_Solver(
            driver= browser, # Your Web Driver
            debug=True,
        )
        solver.solve_recaptcha()
        btn.click()
    except:
        print("no capcha")
        pass

Lấy tất cả các link của bất động sản tại Đà Nẵng

In [ ]:
urlDaNang = "https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/3/da-nang.html/trang--{}.html"
file = open(nameFile, "a+")
browser = webdriver.Chrome(executable_path="./chromedriver.exe")
for pagenum in range(1, 1462):
   try:
      browser.get(urlDaNang.format(pagenum))
      if(len(browser.find_elements(By.CLASS_NAME, 'verify')) > 0):
         try:
            new_thred = threading.Thread(target=resolveCapcha, args=(browser,))
            new_thred.start()
            time.sleep(30)
         except:
            print("Error: unable to start thread")
            pass
      tagAs = browser.find_elements(By.LINK_TEXT, '<< xem chi tiết >>')
      for tagA in tagAs:
         link = tagA.get_attribute("href")
         print(link)
         file.write(link)
         file.write('\n')
         productLinks.append(link)
   except:
      continue
browser.quit()
print(len(productLinks))
file.close()
productLinks

Hàm đọc file to array

In [4]:
def readData(name):
    with open(name, 'r') as f:
        links = []
        for line in f.readlines():
            links.append(str(line))
    return links

Lấy các thuộc tính của bất động sản

In [5]:
detailProductsAloDN = []
browser = webdriver.Chrome(executable_path="chromedriver")
links = readData(nameFile)
for link in links:
    try:
        browser.get(link)
        if(len(browser.find_elements(By.CLASS_NAME, 'verify')) > 0):
            try:
                new_thred = threading.Thread(target=resolveCapcha, args=(browser,))
                new_thred.start()
                time.sleep(30)
            except:
                print("Error: unable to start thread")
                pass
        data = browser.find_element(By.XPATH, "//div[@class='content plp']")
        try:
            name = data.find_element(By.TAG_NAME, "h1").text
            detailProduct = {"name": name}
        except:
            pass
        try:
            price = data.find_element(By.CLASS_NAME, "price").find_element(By.CLASS_NAME, "value").text
            detailProduct["Giá"] = price
        except:
            pass
        try:
            square = data.find_element(By.CLASS_NAME, "square").find_element(By.CLASS_NAME, "value").text
            detailProduct["Diện tích"] = square
        except:
            pass
        try:
            address = data.find_element(By.CLASS_NAME, "address").find_element(By.CLASS_NAME, "value").text
            detailProduct["Địa chỉ"] = address
        except:
            pass
        try:
            infor =  data.find_elements(By.TAG_NAME, "td")
            for i in range(0, len(infor), 2):
                detailProduct[infor[i].text] = infor[i + 1].text if(infor[i + 1].text != '') else 'true'
        except:
            pass
        fileRS = open(nameFileRS, "a+", encoding="utf-8")
        fileRS.write(json.dumps(detailProduct, ensure_ascii=False))
        fileRS.write('\n')
        fileRS.close()
        detailProductsAloDN.append(detailProduct)
        print(detailProduct)
    except Exception as e:
        print(e)
        continue
browser.quit()
detailProductsAloDN

{'name': 'Bán gấp nhà 4 tầng, 6PN, lô góc 2 mặt kiệt 3m - 70m2 đường Trần Cao Vân - Chỉ 4.6 tỷ', 'Giá': '4,6 tỷ', 'Diện tích': '70 m\n2', 'Địa chỉ': 'Đường Trần Cao Vân, Phường Tam Thuận, Quận Thanh Khê, Đà Nẵng', 'Mã tin': '11016540', 'Hướng': '_', 'Phòng ăn': 'true', 'Loại tin': 'Cần bán', 'Đường trước nhà': '3m', 'Nhà bếp': 'true', 'Loại BDS': 'Nhà trong hẻm', 'Pháp lý': 'Sổ hồng/ Sổ đỏ', 'Sân thượng': 'true', 'Chiều ngang': '7m', 'Số lầu': '4', 'Chổ để xe hơi': 'true', 'Chiều dài': '10m', 'Số phòng ngủ': '6', 'Chính chủ': '---'}
{'name': 'Cần Bán Tòa Căn Hộ 5 Tầng - 2 Mặt Tiền - Ngũ Hành Sơn - Gía Rẻ.', 'Giá': '11 tỷ', 'Diện tích': '130 m\n2', 'Địa chỉ': 'Đường Võ Văn Đặng, Phường Hòa Hải, Quận Ngũ Hành Sơn, Đà Nẵng', 'Mã tin': '12343731', 'Hướng': '_', 'Phòng ăn': '---', 'Loại tin': 'Cần bán', 'Đường trước nhà': '7,5m', 'Nhà bếp': '---', 'Loại BDS': 'Nhà hàng, khách sạn', 'Pháp lý': '---', 'Sân thượng': '---', 'Chiều ngang': '9m', 'Số lầu': '5', 'Chổ để xe hơi': '---', 'Chiều dài'

Ghi dữ liệu qua file excel

In [ ]:
csv_columns = []
for product in detailProductsAloDN:
    tempkhac = set(list(product)[0:]) ^ set(csv_columns)
    tempgiong = set(list(product)[0:]) & set(csv_columns)
    csv_columns = tempkhac ^ tempgiong
csv_file = "dataAloNhaDat.csv"
try:
    with open(csv_file, 'w+',encoding= "utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for dataz in detailProductsAloDN:
            try:
                writer.writerow(dataz)
            except:
                continue
except IOError:
    print("I/O error")